In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jan 14 05:50:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# from google.colab import drive
# drive._mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install --upgrade simpletransformers
!pip install --upgrade torch
!pip install --upgrade transformers

     |████████████████████████████████| 247 kB 12.0 MB/s 
     |████████████████████████████████| 1.7 MB 60.0 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 6.8 MB 27.3 MB/s 
     |████████████████████████████████| 306 kB 54.8 MB/s 
     |████████████████████████████████| 3.4 MB 49.6 MB/s 
     |████████████████████████████████| 9.3 MB 38.5 MB/s 
     |████████████████████████████████| 1.2 MB 59.5 MB/s 
     |████████████████████████████████| 895 kB 59.4 MB/s 
     |████████████████████████████████| 3.3 MB 49.5 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 62.4 MB/s 
     |████████████████████████████████| 142 kB 62.3 MB/s 
     |████████████████████████████████| 180 kB 67.9 MB/s 
     |████████████████████████████████| 97 kB 7.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 133 kB 62.7 MB/s 
     |████████████████

     |██████████████████████████████▎ | 834.1 MB 41.4 MB/s eta 0:00:02tcmalloc: large alloc 1147494400 bytes == 0x55e08b3e2000 @  0x7f5bdbca5615 0x55e0257a04cc 0x55e02588047a 0x55e0257a32ed 0x55e025894e1d 0x55e025816e99 0x55e0258119ee 0x55e0257a4bda 0x55e025816d00 0x55e0258119ee 0x55e0257a4bda 0x55e025813737 0x55e025895c66 0x55e025812daf 0x55e025895c66 0x55e025812daf 0x55e025895c66 0x55e025812daf 0x55e0257a5039 0x55e0257e8409 0x55e0257a3c52 0x55e025816c25 0x55e0258119ee 0x55e0257a4bda 0x55e025813737 0x55e0258119ee 0x55e0257a4bda 0x55e025812915 0x55e0257a4afa 0x55e025812c0d 0x55e0258119ee
     |████████████████████████████████| 881.9 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tor

**Restart the notebook after installing above packages**

In [4]:
!unzip '/content/Participants_Data_DCW.zip'

Archive:  /content/Participants_Data_DCW.zip
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     
  inflating: train.csv               
  inflating: __MACOSX/._train.csv    
  inflating: submission.csv          
  inflating: __MACOSX/._submission.csv  


In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from transformers import (AutoModel,AutoModelForMaskedLM, 
                          AutoTokenizer, LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

from collections import Counter
from tqdm import tqdm
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch
pd.options.display.max_colwidth = 200

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
seed_all(13)

In [2]:
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")

In [3]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

train_data['Review'] = train_data['Review'].map(lambda com : clean_text(com))
train_data["Review"] = train_data['Review'].str.replace('[^\w\s]','')

test_data['Review'] = test_data['Review'].map(lambda com : clean_text(com))
test_data["Review"] = test_data['Review'].str.replace('[^\w\s]','')


In [4]:
data = pd.concat([train_data,test_data])
data['Review'] = data['Review'].apply(lambda x: x.replace('\n',''))

text  = '\n'.join(data.Review.str.lower().tolist())

with open('text.txt','w') as f:
    f.write(text)

In [5]:
model_name = 'activebus/BERT-XD_Review'
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained('/content/activebus_bert_final')

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

('/content/activebus_bert_final/tokenizer_config.json',
 '/content/activebus_bert_final/special_tokens_map.json',
 '/content/activebus_bert_final/vocab.txt',
 '/content/activebus_bert_final/added_tokens.json',
 '/content/activebus_bert_final/tokenizer.json')

In [6]:
train_data['Review'].apply(lambda x:tokenizer.tokenize(x)).str.len().describe([0.8,0.9,0.95,0.99])

count    6136.000000
mean       32.192308
std        35.578546
min         2.000000
50%        20.000000
80%        47.000000
90%        73.000000
95%        99.250000
99%       172.650000
max       407.000000
Name: Review, dtype: float64

In [7]:
test_data['Review'].apply(lambda x:tokenizer.tokenize(x)).str.len().describe([0.8,0.9,0.95,0.99])

Token indices sequence length is longer than the specified maximum sequence length for this model (730 > 512). Running this sequence through the model will result in indexing errors


count    2631.000000
mean       32.530597
std        39.368906
min         2.000000
50%        20.000000
80%        48.000000
90%        73.000000
95%        97.000000
99%       162.100000
max       730.000000
Name: Review, dtype: float64

In [8]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention train text file here
    block_size=160)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="text.txt", #mention valid text file here
    block_size=160)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="uhack_sentiments", #select model path for checkpoint
    overwrite_output_dir=True,
    num_train_epochs=25,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy= 'steps',
    save_total_limit=2,
    # eval_steps=200,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    load_best_model_at_end =True,
    prediction_loss_only=True,
    report_to='none')

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset)
trainer.train()
trainer.save_model('/content/activebus_bert_final')


***** Running training *****
  Num examples = 8767
  Num Epochs = 25
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 13700


Step,Training Loss,Validation Loss
500,2.333600,1.996112
1000,2.178800,1.895074
1500,2.114500,1.818793
2000,2.034000,1.783606
2500,1.966600,1.706522
3000,1.924200,1.655064
3500,1.870200,1.610308
4000,1.816200,1.554853
4500,1.757100,1.504841
5000,1.729100,1.492055


***** Running Evaluation *****
  Num examples = 8767
  Batch size = 16
Saving model checkpoint to uhack_sentiments/checkpoint-500
Configuration saved in uhack_sentiments/checkpoint-500/config.json
Model weights saved in uhack_sentiments/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 8767
  Batch size = 16
Saving model checkpoint to uhack_sentiments/checkpoint-1000
Configuration saved in uhack_sentiments/checkpoint-1000/config.json
Model weights saved in uhack_sentiments/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 8767
  Batch size = 16
Saving model checkpoint to uhack_sentiments/checkpoint-1500
Configuration saved in uhack_sentiments/checkpoint-1500/config.json
Model weights saved in uhack_sentiments/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [uhack_sentiments/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 8767
  Batch size = 16
Saving model checkpoint 

In [9]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import sys

import re
import math
import random
import time
import warnings

from torch.optim.swa_utils import AveragedModel, SWALR
    
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers as T
from sklearn.metrics import fbeta_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")
from transformers import AutoTokenizer, AutoModel, AdamW,AutoModelForSequenceClassification
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [10]:
DATA_DIR = "/content/"
OUTPUT_DIR = "/content/"
MODEL_NAME="/content/activebus_bert_final"
TARGETS=['Delivery and Customer Support',	'Dimensions','Functionality','Price',	'Quality','Polarity']

In [11]:
for Target in tqdm(TARGETS):
    def init_logger(log_file=OUTPUT_DIR + "train.log"):
        from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=log_file)
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = init_logger()

    def seed_torch(seed=13):
        random.seed(seed)
        os.environ["PYTHONHASHSEED"] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

    seed = 13
    seed_torch(seed)

    
    train = pd.read_csv(DATA_DIR + "train.csv",usecols=['Review',Target])
    test = pd.read_csv(DATA_DIR + "test.csv",usecols=['Review',Target])
    train['Review']=train['Review'].str.lower()
    test['Review']=test['Review'].str.lower()
    sub = pd.read_csv("/content/submission.csv")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    def get_train_data(train):

        Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        for n, (train_index, val_index) in enumerate(Fold.split(train, train[Target])):
            train.loc[val_index, "fold"] = int(n)
        train["fold"] = train["fold"].astype(np.uint8)

        return train

    def get_test_data(test):
        return test

    train = get_train_data(train)

    class BaseDataset(Dataset):
        def __init__(self, df, model_name, include_labels=True):
            tokenizer = T.BertTokenizer.from_pretrained(model_name)

            self.df = df
            self.include_labels = include_labels

            self.title = df["Review"].tolist()
            self.encoded = tokenizer.batch_encode_plus(
                self.title,
                padding = 'max_length',            
                max_length = 200,
                truncation = True,
                return_attention_mask=True
            )
            
            if self.include_labels:
                self.labels = df[Target].values

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            input_ids = torch.tensor(self.encoded['input_ids'][idx])
            attention_mask = torch.tensor(self.encoded['attention_mask'][idx])

            if self.include_labels:
                label = torch.tensor(self.labels[idx]).float()
                return input_ids, attention_mask, label

            return input_ids, attention_mask

    class BaseModel(nn.Module):
        def __init__(self, model_name):
            super().__init__()

            self.model = T.BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
            self.sigmoid = nn.Sigmoid()

        def forward(self, input_ids, attention_mask):
            out = self.model(input_ids=input_ids, attention_mask=attention_mask)
            out = self.sigmoid(out.logits).squeeze()

            return out

    class AverageMeter(object):
        """Computes and stores the average and current value"""

        def __init__(self):
            self.reset()

        def reset(self):
            self.val = 0
            self.avg = 0
            self.sum = 0
            self.count = 0

        def update(self, val, n=1):
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count


    def asMinutes(s):
        m = math.floor(s / 60)
        s -= m * 60
        return "%dm %ds" % (m, s)


    def timeSince(since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

    def train_fn(train_loader, model, criterion, optimizer, epoch, device):
        start = end = time.time()
        losses = AverageMeter()

        # switch to train mode
        model.train()

        for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
            optimizer.zero_grad()

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            batch_size = labels.size(0)

            y_preds = model(input_ids, attention_mask)

            loss = criterion(y_preds, labels)

            # record loss
            losses.update(loss.item(), batch_size)
            loss.backward()

            optimizer.step()

            if step % 100 == 0 or step == (len(train_loader) - 1):
                print(
                    f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                    f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                    f"Loss: {losses.avg:.4f} "
                )

        return losses.avg

    def valid_fn(valid_loader, model, criterion, device):
        start = end = time.time()
        losses = AverageMeter()

        # switch to evaluation mode
        model.eval()
        preds = []

        for step, (input_ids, attention_mask, labels) in enumerate(valid_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            batch_size = labels.size(0)
            # compute loss
            with torch.no_grad():
                y_preds = model(input_ids, attention_mask)
            
            loss = criterion(y_preds, labels)
            losses.update(loss.item(), batch_size)

            # record score
            preds.append(y_preds.to("cpu").numpy())

            if step % 100 == 0 or step == (len(valid_loader) - 1):
                print(
                    f"EVAL: [{step}/{len(valid_loader)}] "
                    f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                    f"Loss: {losses.avg:.4f} "
                )

        predictions = np.concatenate(preds)
        return losses.avg, predictions

    def inference():
        predictions = []

        test_dataset = BaseDataset(test, MODEL_NAME, include_labels=False)
        test_loader = DataLoader(
            test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True
        )

        for fold in range(5):
            LOGGER.info(f"========== model: bert-base-uncased fold: {fold} inference ==========")
            model = BaseModel(MODEL_NAME)
            model.to(device)
            model.load_state_dict(torch.load(OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth")["model"])
            model.eval()
            preds = []
            for i, (input_ids, attention_mask) in tqdm(enumerate(test_loader), total=len(test_loader)):
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                with torch.no_grad():
                    y_preds = model(input_ids, attention_mask)
                preds.append(y_preds.to("cpu").numpy())
            preds = np.concatenate(preds)
            predictions.append(preds)
        predictions = np.mean(predictions, axis=0)

        return predictions

    def train_loop(train, fold):

        LOGGER.info(f"========== fold: {fold} training ==========")

        # ====================================================
        # Data Loader
        # ====================================================
        trn_idx = train[train["fold"] != fold].index
        val_idx = train[train["fold"] == fold].index

        train_folds = train.loc[trn_idx].reset_index(drop=True)
        valid_folds = train.loc[val_idx].reset_index(drop=True)

        train_dataset = BaseDataset(train_folds,MODEL_NAME)
        valid_dataset = BaseDataset(valid_folds, MODEL_NAME)

        train_loader = DataLoader(
            train_dataset,
            batch_size=16,
            shuffle=True,
            num_workers=4,
            pin_memory=True,
            drop_last=True,
        )
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=16,
            shuffle=False,
            num_workers=4,
            pin_memory=True,
            drop_last=False,
        )

        # ====================================================
        # Model
        # ====================================================
        model = BaseModel(MODEL_NAME)
        model.to(device)

        optimizer = T.AdamW(model.parameters(),lr=2e-5)

        criterion = nn.BCELoss()

        # ====================================================
        # Loop
        # ====================================================
        best_score = 100
        best_loss = np.inf

        for epoch in range(5):
            start_time = time.time()
            
            # train
            avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, device)

            # eval
            avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
            valid_labels = valid_folds[Target].values

            # scoring
            score = log_loss(valid_labels, preds)
            print(f"Log loss for {epoch} is {score}")
            elapsed = time.time() - start_time
            LOGGER.info(
                f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
            )
            LOGGER.info(f"Epoch {epoch+1} - Score: {score}")

            if score < best_score:
                best_score = score
                LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
                torch.save(
                    {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth"
                )

        check_point = torch.load(OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth")

        valid_folds["preds"] = check_point["preds"]

        return valid_folds

    from sklearn.metrics import log_loss
    def get_result(result_df):
        preds = result_df["preds"].values
        labels = result_df[Target].values
        score = log_loss(labels, preds)
        LOGGER.info(f"Score: {score:<.5f}")

    def main():
        # Training
        oof_df = pd.DataFrame()
        for fold in range(5):
            _oof_df = train_loop(train, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)
            
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        
        # Save OOF result
        oof_df.to_csv(OUTPUT_DIR + "oof_df.csv", index=False)

        # Inference
        predictions = inference()
        
        # submission
        sub[Target] = predictions
        sub.to_csv('/content/submission.csv', index=False)

    if __name__ == "__main__":
        main()

  0%|          | 0/6 [00:00<?, ?it/s]Didn't find file /content/activebus_bert_final/added_tokens.json. We won't load it.
loading file /content/activebus_bert_final/vocab.txt
loading file /content/activebus_bert_final/tokenizer.json
loading file None
loading file /content/activebus_bert_final/special_tokens_map.json
loading file /content/activebus_bert_final/tokenizer_config.json
========== fold: 0 training ==========
Didn't find file /content/activebus_bert_final/added_tokens.json. We won't load it.
loading file /content/activebus_bert_final/vocab.txt
loading file None
loading file /content/activebus_bert_final/special_tokens_map.json
loading file /content/activebus_bert_final/tokenizer_config.json
loading file /content/activebus_bert_final/tokenizer.json
Didn't find file /content/activebus_bert_final/added_tokens.json. We won't load it.
loading file /content/activebus_bert_final/vocab.txt
loading file None
loading file /content/activebus_bert_final/special_tokens_map.json
loading file

Epoch: [1][0/306] Elapsed 0m 0s (remain 3m 4s) Loss: 0.6982 
Epoch: [1][100/306] Elapsed 0m 33s (remain 1m 7s) Loss: 0.1538 
Epoch: [1][200/306] Elapsed 1m 5s (remain 0m 34s) Loss: 0.0983 
Epoch: [1][300/306] Elapsed 1m 37s (remain 0m 1s) Loss: 0.0811 
Epoch: [1][305/306] Elapsed 1m 39s (remain 0m 0s) Loss: 0.0808 
EVAL: [0/77] Elapsed 0m 0s (remain 0m 23s) Loss: 0.0190 


Epoch 1 - avg_train_loss: 0.0808  avg_val_loss: 0.0433  time: 108s
Epoch 1 - Score: 0.043295294722900134
Epoch 1 - Save Best Score: 0.0433 Model


EVAL: [76/77] Elapsed 0m 8s (remain 0m 0s) Loss: 0.0433 
Log loss for 0 is 0.043295294722900134
Epoch: [2][0/306] Elapsed 0m 0s (remain 2m 52s) Loss: 0.0268 
Epoch: [2][100/306] Elapsed 0m 33s (remain 1m 7s) Loss: 0.0256 
Epoch: [2][200/306] Elapsed 1m 5s (remain 0m 34s) Loss: 0.0253 
Epoch: [2][300/306] Elapsed 1m 37s (remain 0m 1s) Loss: 0.0304 
Epoch: [2][305/306] Elapsed 1m 39s (remain 0m 0s) Loss: 0.0300 
EVAL: [0/77] Elapsed 0m 0s (remain 0m 24s) Loss: 0.0025 


Epoch 2 - avg_train_loss: 0.0300  avg_val_loss: 0.0256  time: 108s
Epoch 2 - Score: 0.02563582293416983
Epoch 2 - Save Best Score: 0.0256 Model


...

Epoch 4 - avg_train_loss: 0.0252  avg_val_loss: 0.1407  time: 108s
Epoch 4 - avg_train_loss: 0.0252  avg_val_loss: 0.1407  time: 108s
Epoch 4 - avg_train_loss: 0.0252  avg_val_loss: 0.1407  time: 108s
Epoch 4 - avg_train_loss: 0.0252  avg_val_loss: 0.1407  time: 108s
Epoch 4 - avg_train_loss: 0.0252  avg_val_loss: 0.1407  time: 108s
Epoch 4 - avg_train_loss: 0.0252  avg_val_loss: 0.1407  time: 108s
Epoch 4 - Score: 0.14066506236158394
Epoch 4 - Score: 0.14066506236158394
Epoch 4 - Score: 0.14066506236158394
Epoch 4 - Score: 0.14066506236158394
Epoch 4 - Score: 0.14066506236158394
Epoch 4 - Score: 0.14066506236158394


EVAL: [76/77] Elapsed 0m 8s (remain 0m 0s) Loss: 0.1407 
Log loss for 3 is 0.14066506236158394
Epoch: [5][0/306] Elapsed 0m 0s (remain 3m 3s) Loss: 0.0080 
Epoch: [5][100/306] Elapsed 0m 33s (remain 1m 7s) Loss: 0.0135 
Epoch: [5][200/306] Elapsed 1m 5s (remain 0m 34s) Loss: 0.0169 
Epoch: [5][300/306] Elapsed 1m 38s (remain 0m 1s) Loss: 0.0170 
Epoch: [5][305/306] Elapsed 1m 39s (remain 0m 0s) Loss: 0.0181 
EVAL: [0/77] Elapsed 0m 0s (remain 0m 26s) Loss: 0.0046 


Epoch 5 - avg_train_loss: 0.0181  avg_val_loss: 0.2122  time: 108s
Epoch 5 - avg_train_loss: 0.0181  avg_val_loss: 0.2122  time: 108s
Epoch 5 - avg_train_loss: 0.0181  avg_val_loss: 0.2122  time: 108s
Epoch 5 - avg_train_loss: 0.0181  avg_val_loss: 0.2122  time: 108s
Epoch 5 - avg_train_loss: 0.0181  avg_val_loss: 0.2122  time: 108s
Epoch 5 - avg_train_loss: 0.0181  avg_val_loss: 0.2122  time: 108s
Epoch 5 - Score: 0.21217511663388042
Epoch 5 - Score: 0.21217511663388042
Epoch 5 - Score: 0.21217511663388042
Epoch 5 - Score: 0.21217511663388042
Epoch 5 - Score: 0.21217511663388042
Epoch 5 - Score: 0.21217511663388042


EVAL: [76/77] Elapsed 0m 8s (remain 0m 0s) Loss: 0.2122 
Log loss for 4 is 0.21217511663388042


========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
Score: 0.10001
Score: 0.10001
Score: 0.10001
Score: 0.10001
Score: 0.10001
Score: 0.10001
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
Score: 0.10947
Score: 0.10947
Score: 0.10947
Score: 0.10947
Score: 0.10947
Score: 0.10947
Didn't find file /content/activebus_bert_final/added_tokens.json. We won't load it.
loading file /content/activebus_bert_final/vocab.txt
loading file None
loading file /content/activebus_bert_final/special_tokens_map.json
loading file /content/activebus_bert_final/tokenizer_config.json
loading file /content/activebus_bert_final/tokenizer.json
========== model: bert-base-uncased fold: 0 inference ==========
========== model: bert-base-uncased fo

In [12]:
TARGETS=['Components',	'Design and Aesthetics',	'Installation']

In [13]:
for Target in tqdm(TARGETS):
    def init_logger(log_file=OUTPUT_DIR + "train.log"):
        from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=log_file)
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = init_logger()

    def seed_torch(seed=13):
        random.seed(seed)
        os.environ["PYTHONHASHSEED"] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

    seed = 13
    seed_torch(seed)

    
    train = pd.read_csv(DATA_DIR + "train.csv",usecols=['Review',Target])
    test = pd.read_csv(DATA_DIR + "test.csv",usecols=['Review',Target])
    train['Review'] = train['Review'].map(lambda x: re.sub(r'\W+', ' ', x))
    test['Review'] = test['Review'].map(lambda x: re.sub(r'\W+', ' ', x))
    train['Review']=train['Review'].str.lower()
    test['Review']=test['Review'].str.lower()
    sub = pd.read_csv("/content/submission.csv")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    


    def get_train_data(train):

        Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        for n, (train_index, val_index) in enumerate(Fold.split(train, train[Target])):
            train.loc[val_index, "fold"] = int(n)
        train["fold"] = train["fold"].astype(np.uint8)

        return train

    def get_test_data(test):
        return test

    train = get_train_data(train)

    class BaseDataset(Dataset):
        def __init__(self, df, model_name, include_labels=True):
            tokenizer = T.BertTokenizer.from_pretrained(model_name)

            self.df = df
            self.include_labels = include_labels

            self.title = df["Review"].tolist()
            self.encoded = tokenizer.batch_encode_plus(
                self.title,
                padding = 'max_length',            
                max_length = 192,
                truncation = True,
                return_attention_mask=True
            )
            
            if self.include_labels:
                self.labels = df[Target].values

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            input_ids = torch.tensor(self.encoded['input_ids'][idx])
            attention_mask = torch.tensor(self.encoded['attention_mask'][idx])

            if self.include_labels:
                label = torch.tensor(self.labels[idx]).float()
                return input_ids, attention_mask, label

            return input_ids, attention_mask

    class BaseModel(nn.Module):
        def __init__(self, model_name):
            super().__init__()

            self.model = T.BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
            self.sigmoid = nn.Sigmoid()

        def forward(self, input_ids, attention_mask):
            out = self.model(input_ids=input_ids, attention_mask=attention_mask)
            out = self.sigmoid(out.logits).squeeze()

            return out

    class AverageMeter(object):
        """Computes and stores the average and current value"""

        def __init__(self):
            self.reset()

        def reset(self):
            self.val = 0
            self.avg = 0
            self.sum = 0
            self.count = 0

        def update(self, val, n=1):
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count


    def asMinutes(s):
        m = math.floor(s / 60)
        s -= m * 60
        return "%dm %ds" % (m, s)


    def timeSince(since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

    def train_fn(train_loader, model, criterion, optimizer, epoch, device):
        start = end = time.time()
        losses = AverageMeter()

        # switch to train mode
        model.train()

        for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
            optimizer.zero_grad()

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            batch_size = labels.size(0)

            y_preds = model(input_ids, attention_mask)

            loss = criterion(y_preds, labels)

            # record loss
            losses.update(loss.item(), batch_size)
            loss.backward()

            optimizer.step()

            if step % 100 == 0 or step == (len(train_loader) - 1):
                print(
                    f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                    f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                    f"Loss: {losses.avg:.4f} "
                )

        return losses.avg

    def valid_fn(valid_loader, model, criterion, device):
        start = end = time.time()
        losses = AverageMeter()

        # switch to evaluation mode
        model.eval()
        preds = []

        for step, (input_ids, attention_mask, labels) in enumerate(valid_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            batch_size = labels.size(0)
            # compute loss
            with torch.no_grad():
                y_preds = model(input_ids, attention_mask)
            
            loss = criterion(y_preds, labels)
            losses.update(loss.item(), batch_size)

            # record score
            preds.append(y_preds.to("cpu").numpy())

            if step % 100 == 0 or step == (len(valid_loader) - 1):
                print(
                    f"EVAL: [{step}/{len(valid_loader)}] "
                    f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                    f"Loss: {losses.avg:.4f} "
                )

        predictions = np.concatenate(preds)
        return losses.avg, predictions

    def inference():
        predictions = []

        test_dataset = BaseDataset(test, MODEL_NAME, include_labels=False)
        test_loader = DataLoader(
            test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True
        )

        for fold in range(5):
            LOGGER.info(f"========== model: bert-base-uncased fold: {fold} inference ==========")
            model = BaseModel(MODEL_NAME)
            model.to(device)
            model.load_state_dict(torch.load(OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth")["model"])
            model.eval()
            preds = []
            for i, (input_ids, attention_mask) in tqdm(enumerate(test_loader), total=len(test_loader)):
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                with torch.no_grad():
                    y_preds = model(input_ids, attention_mask)
                preds.append(y_preds.to("cpu").numpy())
            preds = np.concatenate(preds)
            predictions.append(preds)
        predictions = np.mean(predictions, axis=0)

        return predictions

    def train_loop(train, fold):

        LOGGER.info(f"========== fold: {fold} training ==========")

        # ====================================================
        # Data Loader
        # ====================================================
        trn_idx = train[train["fold"] != fold].index
        val_idx = train[train["fold"] == fold].index

        train_folds = train.loc[trn_idx].reset_index(drop=True)
        valid_folds = train.loc[val_idx].reset_index(drop=True)

        train_dataset = BaseDataset(train_folds,MODEL_NAME)
        valid_dataset = BaseDataset(valid_folds, MODEL_NAME)

        train_loader = DataLoader(
            train_dataset,
            batch_size=16,
            shuffle=True,
            num_workers=4,
            pin_memory=True,
            drop_last=True,
        )
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=16,
            shuffle=False,
            num_workers=4,
            pin_memory=True,
            drop_last=False,
        )

        # ====================================================
        # Model
        # ====================================================
        model = BaseModel(MODEL_NAME)
        model.to(device)

        optimizer = T.AdamW(model.parameters(),lr=2e-5)

        criterion = nn.BCELoss()

        # ====================================================
        # Loop
        # ====================================================
        best_score = 100
        best_loss = np.inf

        for epoch in range(5):
            start_time = time.time()
            
            # train
            avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, device)

            # eval
            avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
            valid_labels = valid_folds[Target].values

            # scoring
            score = log_loss(valid_labels, preds)
            print(f"Log loss for {epoch} is {score}")
            elapsed = time.time() - start_time
            LOGGER.info(
                f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
            )
            LOGGER.info(f"Epoch {epoch+1} - Score: {score}")

            if score < best_score:
                best_score = score
                LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
                torch.save(
                    {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth"
                )

        check_point = torch.load(OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth")

        valid_folds["preds"] = check_point["preds"]

        return valid_folds

    from sklearn.metrics import log_loss
    def get_result(result_df):
        preds = result_df["preds"].values
        labels = result_df[Target].values
        score = log_loss(labels, preds)
        LOGGER.info(f"Score: {score:<.5f}")

    def main():
        # Training
        oof_df = pd.DataFrame()
        for fold in range(5):
            _oof_df = train_loop(train, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)
            
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        
        # Save OOF result
        oof_df.to_csv(OUTPUT_DIR + "oof_df.csv", index=False)

        # Inference
        predictions = inference()
        
        # submission
        sub[Target] = predictions
        sub.to_csv('/content/submission.csv', index=False)

    if __name__ == "__main__":
        main()

  0%|          | 0/3 [00:00<?, ?it/s]Didn't find file /content/activebus_bert_final/added_tokens.json. We won't load it.
loading file /content/activebus_bert_final/vocab.txt
loading file /content/activebus_bert_final/tokenizer.json
loading file None
loading file /content/activebus_bert_final/special_tokens_map.json
loading file /content/activebus_bert_final/tokenizer_config.json
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
Didn't find file /content/activebus_bert_final/added_tokens.json. We won't load it.
loading file /content/activebus_bert_final/vocab.txt
loading file None
loading file /content/activebus_bert_final/special_tokens_map.json
loading file /content/activebus_bert_final/tokenizer_config.json
loading file /content/activebus_bert_final/tokenizer.json


Epoch: [1][0/306] Elapsed 0m 0s (remain 2m 56s) Loss: 0.7026 
Epoch: [1][100/306] Elapsed 0m 31s (remain 1m 3s) Loss: 0.2203 
Epoch: [1][200/306] Elapsed 1m 2s (remain 0m 32s) Loss: 0.1701 
Epoch: [1][300/306] Elapsed 1m 33s (remain 0m 1s) Loss: 0.1530 
Epoch: [1][305/306] Elapsed 1m 34s (remain 0m 0s) Loss: 0.1520 
EVAL: [0/77] Elapsed 0m 0s (remain 0m 25s) Loss: 0.1294 


Epoch 1 - avg_train_loss: 0.1520  avg_val_loss: 0.0830  time: 103s
Epoch 1 - avg_train_loss: 0.1520  avg_val_loss: 0.0830  time: 103s
Epoch 1 - avg_train_loss: 0.1520  avg_val_loss: 0.0830  time: 103s
Epoch 1 - avg_train_loss: 0.1520  avg_val_loss: 0.0830  time: 103s
Epoch 1 - avg_train_loss: 0.1520  avg_val_loss: 0.0830  time: 103s
Epoch 1 - avg_train_loss: 0.1520  avg_val_loss: 0.0830  time: 103s
Epoch 1 - avg_train_loss: 0.1520  avg_val_loss: 0.0830  time: 103s
Epoch 1 - Score: 0.08297779627374892
Epoch 1 - Score: 0.08297779627374892
Epoch 1 - Score: 0.08297779627374892
Epoch 1 - Score: 0.08297779627374892
Epoch 1 - Score: 0.08297779627374892
Epoch 1 - Score: 0.08297779627374892
Epoch 1 - Score: 0.08297779627374892
Epoch 1 - Save Best Score: 0.0830 Model
Epoch 1 - Save Best Score: 0.0830 Model
Epoch 1 - Save Best Score: 0.0830 Model
Epoch 1 - Save Best Score: 0.0830 Model
Epoch 1 - Save Best Score: 0.0830 Model
Epoch 1 - Save Best Score: 0.0830 Model
Epoch 1 - Save Best Score: 0.083

EVAL: [76/77] Elapsed 0m 8s (remain 0m 0s) Loss: 0.0830 
Log loss for 0 is 0.08297779627374892
Epoch: [2][0/306] Elapsed 0m 0s (remain 2m 54s) Loss: 0.0139 
Epoch: [2][100/306] Elapsed 0m 31s (remain 1m 3s) Loss: 0.0688 
Epoch: [2][200/306] Elapsed 1m 2s (remain 0m 32s) Loss: 0.0769 
Epoch: [2][300/306] Elapsed 1m 33s (remain 0m 1s) Loss: 0.0783 
Epoch: [2][305/306] Elapsed 1m 34s (remain 0m 0s) Loss: 0.0801 
EVAL: [0/77] Elapsed 0m 0s (remain 0m 28s) Loss: 0.1317 


Epoch 2 - avg_train_loss: 0.0801  avg_val_loss: 0.0908  time: 103s
Epoch 2 - avg_train_loss: 0.0801  avg_val_loss: 0.0908  time: 103s
Epoch 2 - avg_train_loss: 0.0801  avg_val_loss: 0.0908  time: 103s
Epoch 2 - avg_train_loss: 0.0801  avg_val_loss: 0.0908  time: 103s
Epoch 2 - avg_train_loss: 0.0801  avg_val_loss: 0.0908  time: 103s
Epoch 2 - avg_train_loss: 0.0801  avg_val_loss: 0.0908  time: 103s
Epoch 2 - avg_train_loss: 0.0801  avg_val_loss: 0.0908  time: 103s
Epoch 2 - Score: 0.09075664963900469
Epoch 2 - Score: 0.09075664963900469
Epoch 2 - Score: 0.09075664963900469
Epoch 2 - Score: 0.09075664963900469
Epoch 2 - Score: 0.09075664963900469
Epoch 2 - Score: 0.09075664963900469
Epoch 2 - Score: 0.09075664963900469


...

Epoch 4 - avg_train_loss: 0.0229  avg_val_loss: 0.0942  time: 103s
Epoch 4 - avg_train_loss: 0.0229  avg_val_loss: 0.0942  time: 103s
Epoch 4 - avg_train_loss: 0.0229  avg_val_loss: 0.0942  time: 103s
Epoch 4 - avg_train_loss: 0.0229  avg_val_loss: 0.0942  time: 103s
Epoch 4 - avg_train_loss: 0.0229  avg_val_loss: 0.0942  time: 103s
Epoch 4 - avg_train_loss: 0.0229  avg_val_loss: 0.0942  time: 103s
Epoch 4 - avg_train_loss: 0.0229  avg_val_loss: 0.0942  time: 103s
Epoch 4 - avg_train_loss: 0.0229  avg_val_loss: 0.0942  time: 103s
Epoch 4 - avg_train_loss: 0.0229  avg_val_loss: 0.0942  time: 103s
Epoch 4 - Score: 0.09416210613083592
Epoch 4 - Score: 0.09416210613083592
Epoch 4 - Score: 0.09416210613083592
Epoch 4 - Score: 0.09416210613083592
Epoch 4 - Score: 0.09416210613083592
Epoch 4 - Score: 0.09416210613083592
Epoch 4 - Score: 0.09416210613083592
Epoch 4 - Score: 0.09416210613083592
Epoch 4 - Score: 0.09416210613083592


EVAL: [76/77] Elapsed 0m 8s (remain 0m 0s) Loss: 0.0942 
Log loss for 3 is 0.09416210613083592
Epoch: [5][0/306] Elapsed 0m 0s (remain 2m 59s) Loss: 0.0068 
Epoch: [5][100/306] Elapsed 0m 31s (remain 1m 3s) Loss: 0.0120 
Epoch: [5][200/306] Elapsed 1m 2s (remain 0m 32s) Loss: 0.0141 
Epoch: [5][300/306] Elapsed 1m 33s (remain 0m 1s) Loss: 0.0174 
Epoch: [5][305/306] Elapsed 1m 34s (remain 0m 0s) Loss: 0.0172 
EVAL: [0/77] Elapsed 0m 0s (remain 0m 25s) Loss: 0.2026 


Epoch 5 - avg_train_loss: 0.0172  avg_val_loss: 0.1254  time: 103s
Epoch 5 - avg_train_loss: 0.0172  avg_val_loss: 0.1254  time: 103s
Epoch 5 - avg_train_loss: 0.0172  avg_val_loss: 0.1254  time: 103s
Epoch 5 - avg_train_loss: 0.0172  avg_val_loss: 0.1254  time: 103s
Epoch 5 - avg_train_loss: 0.0172  avg_val_loss: 0.1254  time: 103s
Epoch 5 - avg_train_loss: 0.0172  avg_val_loss: 0.1254  time: 103s
Epoch 5 - avg_train_loss: 0.0172  avg_val_loss: 0.1254  time: 103s
Epoch 5 - avg_train_loss: 0.0172  avg_val_loss: 0.1254  time: 103s
Epoch 5 - avg_train_loss: 0.0172  avg_val_loss: 0.1254  time: 103s
Epoch 5 - Score: 0.12538878357651004
Epoch 5 - Score: 0.12538878357651004
Epoch 5 - Score: 0.12538878357651004
Epoch 5 - Score: 0.12538878357651004
Epoch 5 - Score: 0.12538878357651004
Epoch 5 - Score: 0.12538878357651004
Epoch 5 - Score: 0.12538878357651004
Epoch 5 - Score: 0.12538878357651004
Epoch 5 - Score: 0.12538878357651004


EVAL: [76/77] Elapsed 0m 8s (remain 0m 0s) Loss: 0.1254 
Log loss for 4 is 0.12538878357651004


========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
Score: 0.09144
Score: 0.09144
Score: 0.09144
Score: 0.09144
Score: 0.09144
Score: 0.09144
Score: 0.09144
Score: 0.09144
Score: 0.09144
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
Score: 0.07093
Score: 0.07093
Score: 0.07093
Score: 0.07093
Score: 0.07093
Score: 0.07093
Score: 0.07093
Score: 0.07093
Score: 0.07093
Didn't find file /content/activebus_bert_final/added_tokens.json. We won't load it.
loading file /content/activebus_bert_final/vocab.txt
loading file None
loading file /con

In [14]:
TARGETS=['Features',	'Material','Usability']

In [15]:
for Target in tqdm(TARGETS):
    def init_logger(log_file=OUTPUT_DIR + "train.log"):
        from logging import INFO, FileHandler, Formatter, StreamHandler, getLogger

        logger = getLogger(__name__)
        logger.setLevel(INFO)
        handler1 = StreamHandler()
        handler1.setFormatter(Formatter("%(message)s"))
        handler2 = FileHandler(filename=log_file)
        handler2.setFormatter(Formatter("%(message)s"))
        logger.addHandler(handler1)
        logger.addHandler(handler2)
        return logger

    LOGGER = init_logger()

    def seed_torch(seed=13):
        random.seed(seed)
        os.environ["PYTHONHASHSEED"] = str(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

    seed = 13
    seed_torch(seed)

    train = pd.read_csv(DATA_DIR + "train.csv",usecols=['Review',Target])
    test = pd.read_csv(DATA_DIR + "test.csv",usecols=['Review',Target])
    train['Review']=train['Review'].str.lower()
    test['Review']=test['Review'].str.lower()
    sub = pd.read_csv("/content/submission.csv")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    def get_train_data(train):

        Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        for n, (train_index, val_index) in enumerate(Fold.split(train, train[Target])):
            train.loc[val_index, "fold"] = int(n)
        train["fold"] = train["fold"].astype(np.uint8)

        return train

    def get_test_data(test):
        return test

    train = get_train_data(train)

    class BaseDataset(Dataset):
        def __init__(self, df, model_name, include_labels=True):
            tokenizer = T.BertTokenizer.from_pretrained(model_name)

            self.df = df
            self.include_labels = include_labels

            self.title = df["Review"].tolist()
            self.encoded = tokenizer.batch_encode_plus(
                self.title,
                padding = 'max_length',            
                max_length = 208,
                truncation = True,
                return_attention_mask=True
            )
            
            if self.include_labels:
                self.labels = df[Target].values

        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            input_ids = torch.tensor(self.encoded['input_ids'][idx])
            attention_mask = torch.tensor(self.encoded['attention_mask'][idx])

            if self.include_labels:
                label = torch.tensor(self.labels[idx]).float()
                return input_ids, attention_mask, label

            return input_ids, attention_mask

    class BaseModel(nn.Module):
        def __init__(self, model_name):
            super().__init__()

            self.model = T.BertForSequenceClassification.from_pretrained(model_name, num_labels=1)
            self.sigmoid = nn.Sigmoid()

        def forward(self, input_ids, attention_mask):
            out = self.model(input_ids=input_ids, attention_mask=attention_mask)
            out = self.sigmoid(out.logits).squeeze()

            return out

    class AverageMeter(object):
        """Computes and stores the average and current value"""

        def __init__(self):
            self.reset()

        def reset(self):
            self.val = 0
            self.avg = 0
            self.sum = 0
            self.count = 0

        def update(self, val, n=1):
            self.val = val
            self.sum += val * n
            self.count += n
            self.avg = self.sum / self.count


    def asMinutes(s):
        m = math.floor(s / 60)
        s -= m * 60
        return "%dm %ds" % (m, s)


    def timeSince(since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return "%s (remain %s)" % (asMinutes(s), asMinutes(rs))

    def train_fn(train_loader, model, criterion, optimizer, epoch, device):
        start = end = time.time()
        losses = AverageMeter()

        # switch to train mode
        model.train()

        for step, (input_ids, attention_mask, labels) in enumerate(train_loader):
            optimizer.zero_grad()

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            batch_size = labels.size(0)

            y_preds = model(input_ids, attention_mask)

            loss = criterion(y_preds, labels)

            # record loss
            losses.update(loss.item(), batch_size)
            loss.backward()

            optimizer.step()

            if step % 100 == 0 or step == (len(train_loader) - 1):
                print(
                    f"Epoch: [{epoch + 1}][{step}/{len(train_loader)}] "
                    f"Elapsed {timeSince(start, float(step + 1) / len(train_loader)):s} "
                    f"Loss: {losses.avg:.4f} "
                )

        return losses.avg

    def valid_fn(valid_loader, model, criterion, device):
        start = end = time.time()
        losses = AverageMeter()

        # switch to evaluation mode
        model.eval()
        preds = []

        for step, (input_ids, attention_mask, labels) in enumerate(valid_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)
            batch_size = labels.size(0)
            # compute loss
            with torch.no_grad():
                y_preds = model(input_ids, attention_mask)
            
            loss = criterion(y_preds, labels)
            losses.update(loss.item(), batch_size)

            # record score
            preds.append(y_preds.to("cpu").numpy())

            if step % 100 == 0 or step == (len(valid_loader) - 1):
                print(
                    f"EVAL: [{step}/{len(valid_loader)}] "
                    f"Elapsed {timeSince(start, float(step + 1) / len(valid_loader)):s} "
                    f"Loss: {losses.avg:.4f} "
                )

        predictions = np.concatenate(preds)
        return losses.avg, predictions

    def inference():
        predictions = []

        test_dataset = BaseDataset(test, MODEL_NAME, include_labels=False)
        test_loader = DataLoader(
            test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True
        )

        for fold in range(5):
            LOGGER.info(f"========== model: bert-base-uncased fold: {fold} inference ==========")
            model = BaseModel(MODEL_NAME)
            model.to(device)
            model.load_state_dict(torch.load(OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth")["model"])
            model.eval()
            preds = []
            for i, (input_ids, attention_mask) in tqdm(enumerate(test_loader), total=len(test_loader)):
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                with torch.no_grad():
                    y_preds = model(input_ids, attention_mask)
                preds.append(y_preds.to("cpu").numpy())
            preds = np.concatenate(preds)
            predictions.append(preds)
        predictions = np.mean(predictions, axis=0)

        return predictions

    def train_loop(train, fold):

        LOGGER.info(f"========== fold: {fold} training ==========")

        # ====================================================
        # Data Loader
        # ====================================================
        trn_idx = train[train["fold"] != fold].index
        val_idx = train[train["fold"] == fold].index

        train_folds = train.loc[trn_idx].reset_index(drop=True)
        valid_folds = train.loc[val_idx].reset_index(drop=True)

        train_dataset = BaseDataset(train_folds,MODEL_NAME)
        valid_dataset = BaseDataset(valid_folds, MODEL_NAME)

        train_loader = DataLoader(
            train_dataset,
            batch_size=16,
            shuffle=True,
            num_workers=4,
            pin_memory=True,
            drop_last=True,
        )
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=16,
            shuffle=False,
            num_workers=4,
            pin_memory=True,
            drop_last=False,
        )

        # ====================================================
        # Model
        # ====================================================
        model = BaseModel(MODEL_NAME)
        model.to(device)

        optimizer = T.AdamW(model.parameters(),lr=2e-5)

        criterion = nn.BCELoss()

        # ====================================================
        # Loop
        # ====================================================
        best_score = 100
        best_loss = np.inf

        for epoch in range(5):
            start_time = time.time()
            
            # train
            avg_loss = train_fn(train_loader, model, criterion, optimizer, epoch, device)

            # eval
            avg_val_loss, preds = valid_fn(valid_loader, model, criterion, device)
            valid_labels = valid_folds[Target].values

            # scoring
            score = log_loss(valid_labels, preds)
            print(f"Log loss for {epoch} is {score}")
            elapsed = time.time() - start_time
            LOGGER.info(
                f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
            )
            LOGGER.info(f"Epoch {epoch+1} - Score: {score}")

            if score < best_score:
                best_score = score
                LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
                torch.save(
                    {"model": model.state_dict(), "preds": preds}, OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth"
                )

        check_point = torch.load(OUTPUT_DIR + f"bert-base-uncased_fold{fold}_best.pth")

        valid_folds["preds"] = check_point["preds"]

        return valid_folds

    from sklearn.metrics import log_loss
    def get_result(result_df):
        preds = result_df["preds"].values
        labels = result_df[Target].values
        score = log_loss(labels, preds)
        LOGGER.info(f"Score: {score:<.5f}")

    def main():
        # Training
        oof_df = pd.DataFrame()
        for fold in range(5):
            _oof_df = train_loop(train, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)
            
        # CV result
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        
        # Save OOF result
        oof_df.to_csv(OUTPUT_DIR + "oof_df.csv", index=False)

        # Inference
        predictions = inference()
        
        # submission
        sub[Target] = predictions
        sub.to_csv('/content/submission.csv', index=False)

    if __name__ == "__main__":
        main()

  0%|          | 0/3 [00:00<?, ?it/s]Didn't find file /content/activebus_bert_final/added_tokens.json. We won't load it.
loading file /content/activebus_bert_final/vocab.txt
loading file /content/activebus_bert_final/tokenizer.json
loading file None
loading file /content/activebus_bert_final/special_tokens_map.json
loading file /content/activebus_bert_final/tokenizer_config.json
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
========== fold: 0 training ==========
Didn't find file /content/activebus_bert_final/added_tokens.json. We won't load it.
loading file /content/activebus_bert_final/vocab.txt
loading file None
loading file /content/activebus_bert_final/special_tokens_map.json
loadin

Epoch: [1][0/306] Elapsed 0m 0s (remain 3m 10s) Loss: 0.6812 
Epoch: [1][100/306] Elapsed 0m 34s (remain 1m 9s) Loss: 0.2142 
Epoch: [1][200/306] Elapsed 1m 7s (remain 0m 35s) Loss: 0.1841 
Epoch: [1][300/306] Elapsed 1m 40s (remain 0m 1s) Loss: 0.1668 
Epoch: [1][305/306] Elapsed 1m 42s (remain 0m 0s) Loss: 0.1664 
EVAL: [0/77] Elapsed 0m 0s (remain 0m 27s) Loss: 0.0588 


Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - avg_train_loss: 0.1664  avg_val_loss: 0.1021  time: 111s
Epoch 1 - Score: 0.10209462196256734
Epoch 1 - Score: 0.10209462196256734
Epoch 1 - Score: 0.10209462196256734
Epoch 1 - Score: 0.10209462196256734
Epoch 1 - Score: 0.10209462196256734
Epoch 1 - Score: 0.10209462196256734
Epoch 1 - Score: 0.10209462196256734
Epoch 1 - Score: 0.10209462196256734
Epoch 1 - Score: 0.102094621962567

EVAL: [76/77] Elapsed 0m 8s (remain 0m 0s) Loss: 0.1021 
Log loss for 0 is 0.10209462196256734
Epoch: [2][0/306] Elapsed 0m 0s (remain 3m 5s) Loss: 0.0632 
Epoch: [2][100/306] Elapsed 0m 34s (remain 1m 9s) Loss: 0.0807 
Epoch: [2][200/306] Elapsed 1m 7s (remain 0m 35s) Loss: 0.0892 
Epoch: [2][300/306] Elapsed 1m 40s (remain 0m 1s) Loss: 0.0856 
Epoch: [2][305/306] Elapsed 1m 42s (remain 0m 0s) Loss: 0.0853 
EVAL: [0/77] Elapsed 0m 0s (remain 0m 26s) Loss: 0.1449 


Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - avg_train_loss: 0.0853  avg_val_loss: 0.1063  time: 111s
Epoch 2 - Score: 0.10625035424232529
Epoch 2 - Score: 0.10625035424232529
Epoch 2 - Score: 0.10625035424232529
Epoch 2 - Score: 0.10625035424232529
Epoch 2 - Score: 0.10625035424232529
Epoch 2 - Score: 0.10625035424232529
Epoch 2 - Score: 0.10625035424232529
Epoch 2 - Score: 0.10625035424232529
Epoch 2 - Score: 0.106250354242325

...

Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - avg_train_loss: 0.0350  avg_val_loss: 0.1993  time: 112s
Epoch 4 - Score: 0.19928820504924205
Epoch 4 - Score: 0.19928820504924205
Epoch 4 - Score: 0.19928820504924205
Epoch 4 - Score: 0.19928820504924205
Epoch 4 - Score: 0.19928820504924205
Epoch 4 - S

EVAL: [76/77] Elapsed 0m 8s (remain 0m 0s) Loss: 0.1993 
Log loss for 3 is 0.19928820504924205
Epoch: [5][0/306] Elapsed 0m 0s (remain 3m 10s) Loss: 0.0194 
Epoch: [5][100/306] Elapsed 0m 34s (remain 1m 9s) Loss: 0.0104 
Epoch: [5][200/306] Elapsed 1m 7s (remain 0m 35s) Loss: 0.0139 
Epoch: [5][300/306] Elapsed 1m 41s (remain 0m 1s) Loss: 0.0195 
Epoch: [5][305/306] Elapsed 1m 42s (remain 0m 0s) Loss: 0.0199 
EVAL: [0/77] Elapsed 0m 0s (remain 0m 28s) Loss: 0.8257 


Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - avg_train_loss: 0.0199  avg_val_loss: 0.2720  time: 112s
Epoch 5 - Score: 0.2720260888865563
Epoch 5 - Score: 0.2720260888865563
Epoch 5 - Score: 0.2720260888865563
Epoch 5 - Score: 0.2720260888865563
Epoch 5 - Score: 0.2720260888865563
Epoch 5 - Score:

EVAL: [76/77] Elapsed 0m 8s (remain 0m 0s) Loss: 0.2720 
Log loss for 4 is 0.2720260888865563


========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
========== fold: 4 result ==========
Score: 0.16181
Score: 0.16181
Score: 0.16181
Score: 0.16181
Score: 0.16181
Score: 0.16181
Score: 0.16181
Score: 0.16181
Score: 0.16181
Score: 0.16181
Score: 0.16181
Score: 0.16181
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
========== CV ==========
Score: 0.13915
Score: 0.13915
Score: 0.13915
Score: 0.13915
Score: 0.13915
S

In [19]:
sub.loc[sub['Delivery and Customer Support']>0.95,'Delivery and Customer Support']=1
sub.loc[sub['Installation']>0.99,'Installation']=1
sub.loc[sub['Polarity']>0.995,'Polarity']=1

In [20]:
sub.to_csv("final_submission.csv",index=False)

In [21]:
sub

,Components,Delivery and Customer Support,Design and Aesthetics,Dimensions,Features,Functionality,Installation,Material,Price,Quality,Usability,Polarity
0,0.064288,0.004370,0.006367,0.465626,0.003052,0.008083,0.003644,0.684759,0.007097,0.796686,0.006844,0.008028
1,0.003568,0.001271,0.003796,0.004844,0.003948,0.966002,0.002018,0.001610,0.003577,0.025209,0.005528,0.984463
2,0.003849,0.000802,0.004170,0.003153,0.018387,0.011440,0.004288,0.001159,0.001668,0.013245,0.982984,1.000000
3,0.004342,0.001007,0.003264,0.004065,0.003459,0.991359,0.002927,0.001221,0.002014,0.004203,0.011061,1.000000
4,0.003362,0.000911,0.003433,0.004024,0.003178,0.079682,0.002456,0.001270,0.002046,0.927882,0.005519,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2626,0.017728,0.000819,0.004584,0.003877,0.004603,0.008004,0.988757,0.001877,0.001912,0.990025,0.930305,1.000000
2627,0.007733,0.006487,0.103094,0.056577,0.003429,0.021570,0.006162,0.001244,0.011111,0.005483,0.008538,0.910537
2628,0.003074,0.001187,0.003138,0.003783,0.002732,0.976333,0.001824,0.002818,0.001916,0.972331,0.004255,0.994582
2629,0.003634,0.000894,0.914709,0.013211,0.003502,0.003982,0.002336,0.024591,0.001898,0.917942,0.008805,0.896557
